In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table,join
from astropy.io import fits

In [8]:
Table.read('J_A+A_672_A159_tablec4.fits')

LC,Star,Sample,GaiaDR3,_2MASS,EPIC,Per,e_Per,Bmag,e_Bmag,Vmag,e_Vmag,Rmag,e_Rmag,Imag,e_Imag,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,Gmag,e_Gmag,BPmag,e_BPmag,RPmag,e_RPmag,M67,G-RP,G-RP0,e_G-RP,BP-RP,BP-RP0,e_BP-RP,B-V,B-V0,e_B-V,V-K,V-K0,e_V-K,RA_ICRS,e_RA_ICRS,DE_ICRS,e_DE_ICRS,Plx,e_Plx,pmRA,e_pmRA,pmDE,e_pmDE,RV,e_RV,OType,SpType,binary,photbin,MS,HPS,DD,xstamp,ystamp,Kepler,K2sc,Everest,FileName,Simbad,_RA_icrs,_DE_icrs
,,,,,,d,d,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,deg,arcsec,deg,arcsec,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,km / s,km / s,,,,,,,,pix,pix,,,,,,deg,deg
bytes2,int16,bytes1,int64,bytes17,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bytes1,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,bytes13,bytes8,bytes1,bytes1,bytes1,bytes1,bytes1,float32,float32,bytes1,bytes1,bytes1,bytes9,bytes6,float64,float64
LC,1,Y,598903678707639296,J08503611+1137059,211398541,32.3500,1.2300,17.180,--,16.200,--,--,--,15.135,--,14.420,0.030,13.890,0.040,13.820,0.040,15.932,0.003,16.460,0.005,15.248,0.004,M,0.684,0.654,0.005,1.212,1.158,0.007,0.980,0.940,--,2.380,2.267,--,132.7,0.0368,11.6,0.0205,1.1289,0.0426,-10.892,0.046,-2.937,0.034,34.240,0.470,RotV*,--,N,N,Y,N,Y,24.821,387.959,Y,Y,N,0001.fits,Simbad,132.7,11.6
LC,2,N,604893680617395200,,--,3.2024,0.0229,17.717,--,16.760,0.021,--,--,15.580,--,14.764,0.036,14.135,0.042,13.957,0.047,16.306,0.003,16.850,0.009,15.599,0.006,F,0.707,--,0.007,1.250,--,0.011,0.957,--,--,2.803,--,0.052,133.0,0.0479,11.6,0.0244,0.8685,0.0593,1.376,0.058,-1.826,0.043,--,--,V*,--,N,N,Y,N,Y,315.933,391.797,N,N,N,0002.fits,Simbad,133.0,11.6
LC,3,N,604894092934247936,,--,20.6881,1.1305,17.526,--,16.575,0.024,--,--,15.507,--,14.820,0.034,14.299,0.044,14.149,0.053,16.273,0.003,16.794,0.006,15.605,0.004,F,0.668,--,0.005,1.190,--,0.007,0.951,--,--,2.426,--,0.058,133.0,0.0448,11.6,0.0239,0.9615,0.0532,-3.089,0.056,-15.703,0.043,--,--,Star,--,N,N,Y,N,N,314.917,357.607,N,N,N,0003.fits,Simbad,133.0,11.6
LC,4,N,604895948360165888,,211397501,24.2382,0.7252,18.049,--,16.735,--,--,--,15.277,--,14.430,0.029,13.734,0.033,13.651,0.040,16.246,0.003,16.986,0.006,15.414,0.006,M,0.832,0.802,0.006,1.572,1.518,0.008,1.314,1.274,--,3.084,2.971,--,132.8,0.0463,11.6,0.0256,1.2702,0.0543,-10.902,0.057,-2.866,0.044,--,--,RotV*,--,Y,Y,Y,Y,N,188.545,397.717,Y,Y,N,0004.fits,Simbad,132.8,11.6
LC,5,Y,604896498115959296,J08514220+1136081,--,23.5000,1.5000,19.560,--,18.087,--,--,--,16.434,--,15.482,0.050,14.689,0.056,14.700,0.086,17.503,0.003,18.374,0.016,16.592,0.006,M,0.911,0.881,0.007,1.782,1.728,0.017,1.473,1.433,--,3.387,3.274,--,132.9,0.0848,11.6,0.0460,1.2575,0.1019,-11.030,0.105,-3.211,0.078,--,--,Star,--,N,N,Y,N,N,269.302,395.716,N,N,N,0005.fits,Simbad,132.9,11.6
LC,6,N,604896566835438464,J08513769+1137037,211398504,1.4550,0.0500,12.910,0.300,12.344,0.059,--,--,11.777,--,11.550,0.021,11.411,0.022,11.341,0.020,12.221,0.003,12.423,0.003,11.883,0.004,M,0.338,0.308,0.005,0.540,0.486,0.005,0.566,0.526,0.306,1.003,0.890,0.062,132.9,0.0318,11.6,0.0172,1.3773,0.0375,-11.125,0.041,-2.056,0.033,32.200,0.350,BlueStraggler,F2V,N,N,N,N,Y,252.260,382.232,Y,Y,N,0006.fits,Simbad,132.9,11.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LC,131,Y,604971466769552128,,--,27.9074,1.9784,18.081,--,16.983,--,--,--,--,--,14.789,0.034,14.136,0.038,13.930,0.046,16.507,0.003,17.177,0.007,15.723,0.005,M,0.784,0.754,0.006,1.455,1.401,0.009,1.098,1.058,--,3.053,2.940,--,1

In [9]:
Table.read ('J_A+A_644_A16_periods.fits')

EPIC,GaiaDR2,_2MASS,RAJ2000,DEJ2000,__BP-RP_0,__B-V_0,__B-V_calc,Gmag,e_Gmag,BPmag,e_BPmag,RPmag,e_RPmag,Bmag,e_Bmag,Vmag,e_Vmag,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,Per,e_Per,Cat,Ncomp,Object,GyroSample,LC,FileName,Simbad
,,,deg,deg,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,d,d,,,,,,,
int32,int64,bytes17,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,uint8,int16,bytes3,bytes1,bytes2,bytes23,bytes6
218933140,4084645500601105536,J19180938-1752498,289.539124,-17.880541,0.872,0.691,0.692,12.359,0.000,12.764,0.002,11.792,0.002,13.284,0.010,12.516,0.010,11.113,0.024,10.823,0.025,10.736,0.023,20.429,2.5,1,9,MS,Y,LC,epic_218933140_hdu.fits,Simbad
219037489,4087622153460553216,J19144383-1739427,288.682617,-17.661888,0.990,0.833,0.800,13.030,0.001,13.504,0.001,12.414,0.002,14.125,0.010,13.215,0.050,11.632,0.021,11.280,0.022,11.145,0.019,22.810,1.5,1,6,MS,Y,LC,epic_219037489_hdu.fits,Simbad
219141523,4087725262729508736,J19153691-1726070,288.903839,-17.435295,2.270,--,1.504,16.561,0.001,17.813,0.006,15.443,0.015,--,--,--,--,13.914,0.026,13.314,0.028,13.101,0.038,26.891,1.8,1,4,MS,Y,LC,epic_219141523_hdu.fits,Simbad
219238231,4087736159069458304,J19163672-1713101,289.153046,-17.219547,0.783,1.000,0.613,12.091,0.000,12.457,0.001,11.574,0.002,13.373,0.381,12.295,0.216,10.948,0.023,10.684,0.024,10.623,0.021,28.100,0.9,2,3,MS,N,LC,epic_219238231_hdu.fits,Simbad
219275512,4087748833503644800,J19155912-1708032,288.996429,-17.134293,0.884,0.599,0.703,11.859,0.000,12.279,0.001,11.295,0.001,12.544,0.217,11.867,0.166,10.607,0.022,10.283,0.022,10.186,0.021,20.434,0.5,2,6,BIN,N,LC,epic_219275512_hdu.fits,Simbad
219280168,4087714409360276864,J19133741-1707261,288.405884,-17.123953,1.148,0.934,0.945,13.328,0.001,13.886,0.002,12.638,0.003,14.583,0.030,13.572,0.020,11.773,0.023,11.275,0.022,11.172,0.023,23.010,0.8,1,5,MS,Y,LC,epic_219280168_hdu.fits,Simbad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219683737,4183968401721906048,J19175075-1606406,289.461487,-16.111315,1.072,0.913,0.875,13.313,0.001,13.830,0.003,12.658,0.002,14.529,0.020,13.538,0.090,11.841,0.024,11.409,0.022,11.306,0.023,21.756,1.0,1,5,MS,Y,LC,epic_219683737_hdu.fits,Simbad
